# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 9 2023 2:50PM,254589,10,Enova-11178,Emerginnova,Released
1,Jan 9 2023 2:50PM,254589,10,Eme-107193,Emerginnova,Released
2,Jan 9 2023 2:46PM,254588,10,9009933,"Snap Medical Industries, LLC",Released
3,Jan 9 2023 2:46PM,254588,10,9009934,"Snap Medical Industries, LLC",Released
4,Jan 9 2023 2:46PM,254588,10,9009935,"Snap Medical Industries, LLC",Released
5,Jan 9 2023 2:46PM,254588,10,9009936,"Snap Medical Industries, LLC",Released
6,Jan 9 2023 2:46PM,254588,10,9009937,"Snap Medical Industries, LLC",Released
7,Jan 9 2023 2:46PM,254588,10,9009938,"Snap Medical Industries, LLC",Released
8,Jan 9 2023 2:46PM,254588,10,9009939,"Snap Medical Industries, LLC",Released
9,Jan 9 2023 2:46PM,254588,10,9010658,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,254584,Released,1
21,254585,Released,1
22,254587,Released,1
23,254588,Released,13
24,254589,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254584,NaN,NaN,1.0
254585,NaN,NaN,1.0
254587,NaN,NaN,1.0
254588,NaN,NaN,13.0
254589,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254505,9.0,0.0,2.0
254511,0.0,8.0,0.0
254512,0.0,0.0,11.0
254516,2.0,3.0,4.0
254533,2.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254505,9,0,2
254511,0,8,0
254512,0,0,11
254516,2,3,4
254533,2,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254505,9,0,2
1,254511,0,8,0
2,254512,0,0,11
3,254516,2,3,4
4,254533,2,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254505,9,,2
1,254511,,8,
2,254512,,,11
3,254516,2,3,4
4,254533,2,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 9 2023 2:50PM,254589,10,Emerginnova
2,Jan 9 2023 2:46PM,254588,10,"Snap Medical Industries, LLC"
15,Jan 9 2023 2:36PM,254587,19,"GUSA Granules USA, Inc."
16,Jan 9 2023 2:19PM,254585,16,"Emersa Waterbox, LLC"
17,Jan 9 2023 2:05PM,254584,10,Yusen – 3D Matrix
18,Jan 9 2023 1:57PM,254581,10,ISDIN Corporation
30,Jan 9 2023 1:53PM,254580,10,ISDIN Corporation
34,Jan 9 2023 1:49PM,254578,19,Eli Lilly and Company
35,Jan 9 2023 12:03PM,254568,19,"Innogenix, LLC"
59,Jan 9 2023 11:57AM,254567,19,VITABIOTICS USA INC.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 9 2023 2:50PM,254589,10,Emerginnova,,,2
1,Jan 9 2023 2:46PM,254588,10,"Snap Medical Industries, LLC",,,13
2,Jan 9 2023 2:36PM,254587,19,"GUSA Granules USA, Inc.",,,1
3,Jan 9 2023 2:19PM,254585,16,"Emersa Waterbox, LLC",,,1
4,Jan 9 2023 2:05PM,254584,10,Yusen – 3D Matrix,,,1
5,Jan 9 2023 1:57PM,254581,10,ISDIN Corporation,,12,
6,Jan 9 2023 1:53PM,254580,10,ISDIN Corporation,,,4
7,Jan 9 2023 1:49PM,254578,19,Eli Lilly and Company,,,1
8,Jan 9 2023 12:03PM,254568,19,"Innogenix, LLC",,,24
9,Jan 9 2023 11:57AM,254567,19,VITABIOTICS USA INC.,,,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 2:50PM,254589,10,Emerginnova,2,,
1,Jan 9 2023 2:46PM,254588,10,"Snap Medical Industries, LLC",13,,
2,Jan 9 2023 2:36PM,254587,19,"GUSA Granules USA, Inc.",1,,
3,Jan 9 2023 2:19PM,254585,16,"Emersa Waterbox, LLC",1,,
4,Jan 9 2023 2:05PM,254584,10,Yusen – 3D Matrix,1,,
5,Jan 9 2023 1:57PM,254581,10,ISDIN Corporation,,12,
6,Jan 9 2023 1:53PM,254580,10,ISDIN Corporation,4,,
7,Jan 9 2023 1:49PM,254578,19,Eli Lilly and Company,1,,
8,Jan 9 2023 12:03PM,254568,19,"Innogenix, LLC",24,,
9,Jan 9 2023 11:57AM,254567,19,VITABIOTICS USA INC.,10,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 2:50PM,254589,10,Emerginnova,2,,
1,Jan 9 2023 2:46PM,254588,10,"Snap Medical Industries, LLC",13,,
2,Jan 9 2023 2:36PM,254587,19,"GUSA Granules USA, Inc.",1,,
3,Jan 9 2023 2:19PM,254585,16,"Emersa Waterbox, LLC",1,,
4,Jan 9 2023 2:05PM,254584,10,Yusen – 3D Matrix,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 2:50PM,254589,10,Emerginnova,2.0,NaN,NaN
1,Jan 9 2023 2:46PM,254588,10,"Snap Medical Industries, LLC",13.0,NaN,NaN
2,Jan 9 2023 2:36PM,254587,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Jan 9 2023 2:19PM,254585,16,"Emersa Waterbox, LLC",1.0,NaN,NaN
4,Jan 9 2023 2:05PM,254584,10,Yusen – 3D Matrix,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 2:50PM,254589,10,Emerginnova,2.0,0.0,0.0
1,Jan 9 2023 2:46PM,254588,10,"Snap Medical Industries, LLC",13.0,0.0,0.0
2,Jan 9 2023 2:36PM,254587,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Jan 9 2023 2:19PM,254585,16,"Emersa Waterbox, LLC",1.0,0.0,0.0
4,Jan 9 2023 2:05PM,254584,10,Yusen – 3D Matrix,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2036476,27.0,15.0,13.0
15,509129,54.0,4.0,18.0
16,254585,1.0,0.0,0.0
19,1527323,47.0,8.0,0.0
20,509120,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2036476,27.0,15.0,13.0
1,15,509129,54.0,4.0,18.0
2,16,254585,1.0,0.0,0.0
3,19,1527323,47.0,8.0,0.0
4,20,509120,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,27.0,15.0,13.0
1,15,54.0,4.0,18.0
2,16,1.0,0.0,0.0
3,19,47.0,8.0,0.0
4,20,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,27.0
1,15,Released,54.0
2,16,Released,1.0
3,19,Released,47.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,13.0,18.0,0.0,0.0,0.0
Executing,15.0,4.0,0.0,8.0,0.0
Released,27.0,54.0,1.0,47.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,13.0,18.0,0.0,0.0,0.0
1,Executing,15.0,4.0,0.0,8.0,0.0
2,Released,27.0,54.0,1.0,47.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,13.0,18.0,0.0,0.0,0.0
1,Executing,15.0,4.0,0.0,8.0,0.0
2,Released,27.0,54.0,1.0,47.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()